In [9]:
from transformers import DecisionTransformerModel, DecisionTransformerConfig
import numpy as np
import torch

device = "cpu"

scale = 1000.0  # normalization for rewards/returns
TARGET_RETURN = 3600 / scale  # evaluation conditioning targets, 3600 is reasonable from the paper LINK
state_dim = (3, 32, 32)
act_dim = 6

In [58]:
state_dim_flatten = state_dim[0]*state_dim[1]*state_dim[2]

t = 3
target_return = torch.tensor(TARGET_RETURN, device=device, dtype=torch.float32).reshape(1, 1)
states = torch.rand(state_dim, device=device, dtype=torch.float32).reshape(1, 1, state_dim_flatten)
actions = torch.rand((1, act_dim), device=device, dtype=torch.float32)
rewards = torch.rand((1, 1), device=device, dtype=torch.float32)
timesteps = torch.tensor(0, device=device, dtype=torch.long).reshape(1, 1)
attention_mask = torch.zeros(1, 1, device=device, dtype=torch.float32)


print(states.shape)
print(actions.shape)
print(rewards.shape)
print(target_return.shape)

config = DecisionTransformerConfig(state_dim_flatten, act_dim, max_ep_len=4096)


model = DecisionTransformerModel(config).to(device)
model.eval()

state_preds, action_preds, return_preds = model(states=states,
        actions=actions,
        rewards=rewards,
        returns_to_go=target_return,
        timesteps=timesteps,
        attention_mask=attention_mask,
        return_dict=False)

print(torch.argmax(action_preds).item())
print(action_preds.shape)
print(torch.squeeze(action_preds).shape)

state_preds, action_preds, return_preds

torch.Size([1, 1, 3072])
torch.Size([1, 6])
torch.Size([1, 1])
torch.Size([1, 1])
4
torch.Size([1, 1, 6])
torch.Size([6])


(tensor([[[-0.4784, -0.0251, -0.4146,  ...,  0.0184, -0.1038, -0.0374]]],
        grad_fn=<ViewBackward0>),
 tensor([[[-0.0925, -0.1899, -0.1619, -0.3070,  0.2921, -0.1744]]],
        grad_fn=<TanhBackward0>),
 tensor([[[-0.4369]]], grad_fn=<ViewBackward0>))

In [297]:
import random

population = [0, 1, 2, 3, 4]
ls = random.sample(population,5)

print("Pure random: ", ls)

max_sequence = 4

def sample_with_order(population, max_sequence_length):
    k = random.randint(1, max_sequence_length)
    k_start = random.randint(0, len(population)-k)
    return population[k_start: k_start+k]


print(sample_with_order(population, max_sequence))

   

Pure random:  [2, 4, 3, 0, 1]
3
[3]


In [298]:
target_return = torch.ones(1, device=device, dtype=torch.float32).reshape(100, 1)

RuntimeError: shape '[100, 1]' is invalid for input of size 1